In [174]:
!uvicorn main:app

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.2 when using version 1.5.1. This might lead to breaking cod

In [176]:
%%writefile main.py
from fastapi import FastAPI, File, UploadFile
import uvicorn
from pydantic import BaseModel
import pickle 
import pandas as pd
from typing import List
import sklearn
from fastapi.responses import FileResponse

app = FastAPI()

class Item(BaseModel):
    name: str
    year: int
    selling_price: int
    km_driven: int
    fuel: str
    seller_type: str
    transmission: str
    owner: str
    mileage: str
    engine: str
    max_power: str
    torque: str
    seats: float
    
with open("linear_model.pickle", "rb") as f:
    model = pickle.load(f)

@app.post('/predict_item')
def predict(data: Item):
    data = data.dict()
    my_data = pd.DataFrame([data])
    df = my_data.drop(columns = ['torque', 'name', 'selling_price'])
    #km
    df['mileage'] = df['mileage'].str.replace(' kmpl', '').str.replace(' km/kg', '').astype('float')
    #CC
    df['engine'] = df['engine'].str.replace(' CC', '').astype('float')
    #bhp
    df['max_power'] = df['max_power'].str.replace(' bhp', '').astype('float')
    y_pred = model.predict(df)[0]
    return {'price_predict' : y_pred}

@app.post("/predict_items")
def predict_items(file: UploadFile = File(...)):
    df0 = pd.read_csv(file.file)
    df = df0.drop(columns = ['torque', 'name', 'selling_price'])
    #km
    df['mileage'] = df['mileage'].str.replace(' kmpl', '').str.replace(' km/kg', '').astype('float')
    #CC
    df['engine'] = df['engine'].str.replace(' CC', '').astype('float')
    #bhp
    df['max_power'] = df['max_power'].str.replace(' bhp', '').astype('float')
    y_pred = model.predict(df)
    df_final = pd.concat([df0, pd.DataFrame([{'price_predict' : y_pred}])], axis = 1)
    df_final.to_csv("res.csv", index=False)
    return FileResponse("res.csv", media_type="text/csv", filename="res.csv")

Overwriting main.py


In [44]:
!python3 -m pip install "fastapi[all]"

In [132]:
# создаем папку для проекта
!mkdir fastapi_hello_world

# переходим в папку проекта
!cd fastapi_hello_world

# создаем виртуальное окружение
!python3 -m venv .venv

# активируем виртуальное окружение
!source .venv/bin/activate

# ставим модуль `FastAPI`
!python3 -m pip install -U fastapi

# ставим сервер ASGI (uvicorn)
!python3 -m pip install "uvicorn[standard]"

mkdir: fastapi_hello_world: File exists


In [234]:
%%writefile main.py

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}

@app.get("/hello/{name}")
async def say_hello(name: str):
    return {"message": f"Hello {name}"}

Overwriting main.py


In [230]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/Users/nastagavrilova']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [22334] using WatchFiles
INFO:     Started server process [22336]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:57640 - "GET / HTTP/1.1" 200 OK
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [22336]
INFO:     Stopping reloader process [22334]


In [ ]:
    test_data = [[
			data.year, 
			data.km_driven, 
			data.fuel, 
			data.seller_type,
            data.transmission, 
			data.owner, 
			data.mileage, 
			data.engine,
            data.max_power,
            data.seats]]